In [1]:
import os
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='element-deeplabcut', ("Please move to the "
                                                              + "element directory")

In [2]:
import datajoint as dj
from pathlib import Path
import yaml

In [3]:
dj.conn()

[2023-10-12 01:17:45,270][INFO]: Connecting root@fakeservices.datajoint.io:3306
[2023-10-12 01:17:45,278][INFO]: Connected root@fakeservices.datajoint.io:3306


DataJoint connection (connected) root@fakeservices.datajoint.io:3306

In [4]:
from tutorial_pipeline import lab, subject, session, train, model  

[2023-10-12 01:17:45,437][WARNING]: lab.Project and related tables will be removed in a future version of Element Lab. Please use the project schema.


In [ ]:
dj.list_schemas()

In [ ]:
dj.config

In [5]:
from element_interface.utils import find_full_path
data_dir = find_full_path(dj.config['custom']['dlc_root_data_dir'], # root from config
                          dj.config['custom']['current_project_folder'])       


In [6]:
data_dir

PosixPath('/workspaces/element-deeplabcut/example_data/from_top_tracking')

In [7]:
### DLC needs absolute paths
### DataJoint uses relative paths

#dj_config_file -> modify the config uploaded to have the full paths instead, we don't change raw data. 

### DLC Project
dlc_project_path_abs = Path(dj.config["custom"]["dlc_root_data_dir"]) / Path(
    dj.config["custom"]["current_project_folder"]
)  # use pathlib to join; abs path

dlc_project_folder = Path(
    dj.config["custom"]["current_project_folder"]
)  # relative path

### Config file
config_file_abs = dlc_project_path_abs / "config.yaml"  # abs path
assert (
    config_file_abs.exists()
), "Please check the that you have the from_top_tracking folder"

config_file_rel = config_file_abs.relative_to(Path(dj.config["custom"]["dlc_root_data_dir"]).parent)

### Labeled-data
#labeled_data_path_abs = dlc_project_path_abs / "labeled-data" / "train_video"
labeled_data_path_abs = dlc_project_path_abs / "labeled-data" 
labeled_files_rel = os.listdir(labeled_data_path_abs / "train1") 


In [8]:
config_file_rel

PosixPath('example_data/from_top_tracking/config.yaml')

In [9]:
labeled_files_rel

['CollectedData_DJ.csv',
 'CollectedData_DJ.h5',
 'img00674.png',
 'img02507.png',
 'img03417.png',
 'img04243.png',
 'img07087.png',
 'img10817.png',
 'img10854.png',
 'img10997.png',
 'img13211.png',
 'img13538.png',
 'img15154.png',
 'img15553.png',
 'img17741.png',
 'img20640.png',
 'img20796.png',
 'img23499.png',
 'img26252.png',
 'img26419.png',
 'img29448.png',
 'img29883.png',
 'img32079.png',
 'img32500.png',
 'img33628.png',
 'img35209.png',
 'img36905.png',
 'img37720.png',
 'img39434.png',
 'img40578.png',
 'img40733.png',
 'img41989.png',
 'img43673.png',
 'img45967.png',
 'img46726.png',
 'img46858.png',
 'img47791.png',
 'img51645.png',
 'img54116.png',
 'img56508.png',
 'img57346.png',
 'img59317.png']

In [ ]:
(
    dj.Diagram(subject) 
    + dj.Diagram(lab) 
    + dj.Diagram(session) 
    + dj.Diagram(model) 
    + dj.Diagram(train)
)

In [ ]:
dj.Diagram(model) + dj.Diagram(train)

In [10]:
subject.Subject()

subject,subject_nickname,sex,subject_birth_date,subject_description
subject6,,F,2020-01-01,hneih_E105


In [ ]:
# Subject and Session tables
subject.Subject.insert1(
    dict(
        subject="subject6",
        sex="F",
        subject_birth_date="2020-01-01",
        subject_description="hneih_E105",
    ),
    skip_duplicates=True,
)

In [ ]:
#Definition of the dictionary named "session_keys"
session_keys = [
    dict(subject="subject6", session_datetime="2021-06-02 14:04:22"),
    dict(subject="subject6", session_datetime="2021-06-03 14:43:10"),
]

#Insert this dictionary in the Session table
session.Session.insert(session_keys, skip_duplicates=True)
session.Session()

In [ ]:
session.SessionDirectory() #for the worker? is it needed?

In [17]:
# Videoset table 
train.VideoSet.insert1({"video_set_id": 1}, skip_duplicates=True)

for idx, filename in enumerate(labeled_files_rel):
    train.VideoSet.File.insert1(
        {
            "video_set_id": 1, 
            "file_id": idx, 
            "file_path": (dlc_project_folder / "labeled-data" / "train1" / filename).as_posix() 
            #"file_path": dlc_project_folder / "labeled-data" / "train_video" / filename 
        },
        skip_duplicates=True
    )  

In [ ]:
vr_file_insert = dict(subject='test_sub',session_id=1,recording_id=0, file_id=0, file_path='')

In [18]:
model.VideoRecording.File().fetch('file_path')

array([], dtype=object)

In [ ]:
vr_file_insert = dict(subject='test_sub',session_id=3,recording_id=0,file_id=0,file_path='from_top_tracking/videos/test_video.mp4')
model.VideoRecording.File.insert1(vr_file_insert)

In [19]:
model.VideoRecording.File()

subject,session_datetime,recording_id,file_id,"file_path filepath of video, relative to root data directory"


In [ ]:
model.BodyPart.heading


In [27]:
model.Model.insert_new_model(model_name='from_top_tracking_model_test',dlc_config=config_file_rel,
                             shuffle=1,
                             trainingsetindex=0,
                             model_description='Model in example data: from_top_tracking model',
                             paramset_idx=0,
                             params={"snapshotindex":-1,
                                    "project_path":dlc_project_folder})

--- DLC Model specification to be inserted ---
	model_name: from_top_tracking_model_test
	model_description: Model in example data: from_top_tracking model
	scorer: DLCmobnet100fromtoptrackingFeb23shuffle1
	task: from_top_tracking
	date: Feb23
	iteration: 0
	snapshotindex: -1
	shuffle: 1
	trainingsetindex: 0
	project_path: from_top_tracking
	paramset_idx: 0
	-- Template/Contents of config.yaml --
		Task: from_top_tracking
		scorer: DJ
		date: Feb23
		video_sets: {'videos/train1.mp4': {'crop': '0, 500, 0, 500'}}
		bodyparts: ['head', 'bodycenter', 'tailbase']
		start: 0
		stop: 1
		numframes2pick: 20
		pcutoff: 0.6
		dotsize: 3
		alphavalue: 0.7
		colormap: viridis
		TrainingFraction: [0.95]
		iteration: 0
		default_net_type: resnet_50
		snapshotindex: -1
		batch_size: 8
		cropping: False
		x1: 0
		x2: 640
		y1: 277
		y2: 624
		corner2move2: [50, 50]
		move2corner: True
		croppedtraining: None
		default_augmenter: default
		identity: None
		maxiters: 5
		modelprefix: 
		multianimalproje

PermissionError: [Errno 1] Operation not permitted: '/workspaces/element-deeplabcut/example_data/from_top_tracking/dj_dlc_config.yaml'

In [ ]:
model.Model()

In [ ]:
dlc_project_path_abs

In [ ]:
model